In [ ]:
def refresh_token(base_url, refresh_token_id):
        content = 'refresh_token=' + refresh_token_id + '&grant_type=refresh_token'
        response = requests.post(base_url + 'token', data = content, headers={"Content-Type": "application/x-www-form-urlencoded"})
        token_entity = response.json()
        refresh_token = token_entity.get('access_token',token_entity)
        return refresh_token


def request_t_get(host, path, tokenStr):
	return requests.get(host + path, headers={'Authorization': 'bearer ' + tokenStr}).json()


def get_task_group(base_url, token):
        print('GetTaskGroups:')
        url = 'api/taskgroup'
        response = request_t_get(base_url, url, token)
        groups = []
        if 'error' in response:
                if response['error'] == 'success':
                        groups = response['data']
                        for taskgroup in groups:
                                print('%s\t%s'%(taskgroup['taskGroupId'],taskgroup['taskGroupName']))
                else:
                        print(response['error_Description'])
        else:
                print(response)             
        return groups


def get_task_by_group_id(base_url, token, groupId):
        print('GetTasks:')
        url = 'api/task?taskgroupId=' + str(groupId)
        response = request_t_get(base_url, url, token)
        if 'error' in response:
                if response['error'] == 'success':
                        tasks = response['data']
                        for task in tasks:
                                print('%s\t%s'%(task['taskId'],task['taskName']))
                else:
                        print(response['error_Description'])
        else:
                print(response)
        return tasks


def get_data_by_offset(base_url, token, task_id, offset=0, size=10):
        print('GetTaskDataByOffset:')
        url = 'api/allData/getDataOfTaskByOffset?taskId=%s&offset=%s&size=%s'%(task_id, offset, size)
        task_data_result = request_t_get(base_url, url, token)
        show_task_data(task_data_result)
        return task_data_result



def show_task_data(dataResult):
	if 'error' in dataResult:
		if dataResult['error'] == 'success' and 'dataList' in dataResult['data']:
			dataDict = dataResult['data']['dataList'][0]
			for k, v in dataDict.items():
				print("%s\t%s"%(k, v))
		else:
			print(dataResult['error_Description'])
	else:
		print(response)

In [ ]:
import requests

base_url = 'https://dataapi.octoparse.com/'
USERNAME = "YOUR USER NAME"
PASSWORD = "YOUR PASSWORD"

print('Get token:')
content = 'username={0}&password={1}&grant_type=password'.format(USERNAME,PASSWORD)
token_entity = requests.post(base_url + 'token', data = content, headers={"Content-Type": "application/x-www-form-urlencoded"}).json()

print(token_entity)

In [ ]:
refresh_token(base_url, token_entity['refresh_token'] )
token = token_entity['access_token']

In [ ]:
groups = get_task_group(base_url, token)

In [ ]:
tasks = get_task_by_group_id(base_url, token, groups[4]['taskGroupId'])

In [ ]:
task_id = tasks[0]['taskId']

In [ ]:
result = get_data_by_offset(base_url, token, task_id, 0, 30)

In [ ]:
result

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import JsonToolkit, create_json_agent
from langchain_community.tools.json.tool import JsonSpec


In [ ]:
API_KEY = "YOUR API KEY"

llm = ChatOpenAI(model_name="gpt-4o", api_key=API_KEY, temperature=0)
agent = create_json_agent(
    llm=ChatOpenAI(temperature=0, api_key=API_KEY), 
    toolkit=json_toolkit, 
    verbose=True,
    handle_parsing_errors=True
)

agent.run("Summarize the overall sentiments from the comments")

In [ ]:
agent.run("Based on the comments in the dataList, tell me why users dislike the app ")

In [ ]:
agent.run("Based on the comments in the dataList, is there any positive user feedback")